This file is being replaced by 3_3_Alternative

# 3.3 Looking at the Lexical Vocabulary from the Perspective of the Literary Material

In section 3.2 we asked whether we can see differences between Old Babylonian literary compositions in their usage of vocabulary (lemmas and MWEs) attested in the lexical corpus. In this notebook we will change perspective and ask: are there particular lexical texts (or groups of lexical texts) that show a greater engagement with literary vocabulary than others?

In large part, this notebook uses the same techniques and the same code as section 3.2 did, and the reader is referred there for further explanation. In some aspects, however, the process is different.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # this suppresses a warning about pandas from tqdm
import pandas as pd
from ipywidgets import interact
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm.auto import tqdm
tqdm.pandas() # initiate pandas support in tqdm, allowing progress_apply() and progress_map()
import zipfile
import json

Open the file `litlines.p` which was produced in [3_1_Lit_Lex_Vocab.ipynb](./3_1_Lit_Lex_Vocab.ipynb). The file contains the pickled version of the DataFrame `lit_lines` in which the literary ([epsd2/literary](http://oracc.org/epsd2/literary)) corpus is represented in line-by-line format.

In [ ]:
lit_lines = pd.read_pickle('output/litlines.p')
lit_lines

Open the file `lit_lex_vocab` which was produced in [3_2_Lit_Lex.ipynb](./3_2_Lit_Lex.ipynb) and which contains the vocabulary (lemmas and MWEs) shared by lexical and literary texts.

In [ ]:
with open('output/lit_lex_vocab.txt', 'r', encoding = 'utf8') as l:
    lit_lex_vocab = l.read().splitlines()
lit_lex_vocab[:25]

# Read Lexical Corpus
Open the file `lexlines.p` which contains the (Old Babylonian) lexical corpus in line-by-line format. Replace all spaces by underscores, in order to turn lexical entries into Multiple Word Expressions. Remove all lines that contain unlemmatized words.

In [ ]:
lex_lines = pd.read_pickle('output/lexlines.p')
lex_lines['lemma'] = [lemma.replace(' ', '_') for lemma in lex_lines['lemma']]
lex_lines = lex_lines.loc[~lex_lines.lemma.str.contains('\[na\]na')]
lex_lines

### Special Case: OB Nippur Ura 6
The sixth chapter of the Old Babylonian Nippur version of the thematic list Ura deals with foodstuffs and drinks. This chapter was not standardized (each exemplar has its own order of items and sections) and therefore no composite text has been created in [DCCLT](http://oracc.org/dcclt). Instead, the "composite" of [OB Nippur Ura 6](http://oracc.org/dcclt/Q000043) consists of the concatenation of all known Nippur exemplars of the list of foodstuffs. In our current dataframe, therefore, there are no lines where the field `id_text` equals "dcclt/Q000043".

We create a "composite" by changing the field `id_text` in all exemplars of [OB Nippur Ura 6](http://oracc.org/dcclt/Q000043) to "dcclt/Q000043". 

In [ ]:
Ura6 = ["dcclt/P227657",
"dcclt/P227743",
"dcclt/P227791",
"dcclt/P227799",
"dcclt/P227925",
"dcclt/P227927",
"dcclt/P227958",
"dcclt/P227967",
"dcclt/P227979",
"dcclt/P228005",
"dcclt/P228008",
"dcclt/P228200",
"dcclt/P228359",
"dcclt/P228368",
"dcclt/P228488",
"dcclt/P228553",
"dcclt/P228562",
"dcclt/P228663",
"dcclt/P228726",
"dcclt/P228831",
"dcclt/P228928",
"dcclt/P229015",
"dcclt/P229093",
"dcclt/P229119",
"dcclt/P229304",
"dcclt/P229332",
"dcclt/P229350",
"dcclt/P229351",
"dcclt/P229352",
"dcclt/P229353",
"dcclt/P229354",
"dcclt/P229356",
"dcclt/P229357",
"dcclt/P229358",
"dcclt/P229359",
"dcclt/P229360",
"dcclt/P229361",
"dcclt/P229362",
"dcclt/P229365",
"dcclt/P229366",
"dcclt/P229367",
"dcclt/P229890",
"dcclt/P229925",
"dcclt/P230066",
"dcclt/P230208",
"dcclt/P230230",
"dcclt/P230530",
"dcclt/P230586",
"dcclt/P231095",
"dcclt/P231128",
"dcclt/P231424",
"dcclt/P231446",
"dcclt/P231453",
"dcclt/P231458",
"dcclt/P231742",
"dcclt/P266520"]
lex_lines.loc[lex_lines["id_text"].isin(Ura6), "id_text"] = "dcclt/Q000043"

In [ ]:
lex_comp = lex_lines.groupby(['id_text']).agg({'lemma': ' '.join}).reset_index()
lex_comp

# Remove Duplicates
Since the lexical data are drawn from multiple (sub)projects, it is possible that there are duplicates. Duplicates have the same P, Q, or X number. We select the version with the largest number of (lemmatized) words and drop others.

First the field `id_text` is reduced to only the last 7 positions (P, Q, or X, followed by six digits). The new field `length` represents the length of a document in terms of lemmatized words or entries (unlemmatized words and entries have already been dropped above). The DataFrame is then ordered by length (from large to small) and, if duplicate `text_id`s are found, only the first one is kept with the Pandas method `drop_duplicates()`.

In [ ]:
lex_comp['id_text'] = [i[-7:] for i in lex_comp['id_text']]
lex_comp['length'] = [len(lem.split()) for lem in lex_comp['lemma']]
lex_comp = lex_comp.sort_values(by = 'length', ascending = False)
lex_comp = lex_comp.drop_duplicates(subset = 'id_text', keep = 'first')
lex_comp

In [ ]:
cv = CountVectorizer(preprocessor = lambda x: x, tokenizer = lambda x: x.split(), vocabulary = lit_lex_vocab)
dtm = cv.fit_transform(lex_comp['lemma'])
lex_df = pd.DataFrame(dtm.toarray(), columns= cv.get_feature_names(), index=lex_comp["id_text"])

Confusion: lit_lex_vocab, saved form notebook 3.2 includes all lemmas and all MWEs shared by lit and lex. In the data, countvectorizer only sees the lexical *entries* because they have been connected by underscores. Perhaps: do not connect, CountVectorizer() with ngram_range = (1,4) and use CV on lexical *lines* then combine lines to compositions in DTM. No that didn't work.
Other solution

In [ ]:
lex_df.shape

In [ ]:
lex_df = lex_df.loc[: , lex_df.sum(axis=0) > 0].copy()
vocab = lex_df.columns

In [ ]:
lex_df["n_matches"] = lex_df[vocab].astype(bool).sum(axis = 1, numeric_only=True)

In [ ]:
lex_df

In [ ]:
# First get the metadata. 
cat = {}
for proj in ['dcclt', 'dcclt/signlists', 'dcclt/nineveh', 'dcclt/ebla']:
    f = proj.replace('/', '-')
    file = f"jsonzip/{f}.zip" # The ZIP file was downloaded in notebook 3_1
    z = zipfile.ZipFile(file) 
    st = z.read(f"{proj}/catalogue.json").decode("utf-8")
    j = (json.loads(st))
    cat.update(j["members"])
cat_df = pd.DataFrame(cat).T
cat_df["id_text"] = cat_df["id_text"].fillna(cat_df["id_composite"])
cat_df = cat_df.fillna('')
cat_df = cat_df[["id_text", "designation", "subgenre"]]

In [ ]:
lex = pd.merge(cat_df, lex_df['n_matches'], on = 'id_text', how = 'inner')
lex = pd.merge(lex, lex_comp[['length', 'id_text']], on = 'id_text', how = 'inner')

In [ ]:
lex['norm'] = lex['n_matches'] / lex['length']
lex = lex.sort_values(by = 'norm', ascending = False)
lex.loc[lex.length > 250]

In [ ]:
anchor = '<a href="http://oracc.org/dcclt/{}", target="_blank">{}</a>'
lex2 = lex.copy()
lex2['id_text'] = [anchor.format(val,val) for val in lex['id_text']]

In [ ]:
@interact(sort_by = lex2.columns, rows = (1, len(lex2), 1), min_length = (1,500,5))
def sort_df(sort_by = "norm", ascending = False, rows = 25, min_length = 250):
    return lex2.loc[lex2.length >= min_length].sort_values(by = sort_by, ascending = ascending).reset_index(drop=True)[:rows].style

Next step: look at important words with tfidf.

Note: first make ngrams (as above) then TfidfVectorizer() with vocabulary.

In [ ]:
lit_comp2 = lit_lines.groupby(['id_text']).agg({'lemma' : ' '.join}).reset_index()
lit_comp2['id_text'] = [i[-7:] for i in lit_comp2['id_text']]

In [ ]:
tv = TfidfVectorizer(token_pattern = r'[^ ]+', ngram_range = (1,5), vocabulary = vocab)
dtm = tv.fit_transform(lit_comp2['lemma'])
lit_df = pd.DataFrame(dtm.toarray(), columns= tv.get_feature_names(), index=lit_comp2["id_text"])
#cols = [col for col in lit_df.columns if not '[na]na' in col]
#lit_df = lit_df[cols]

In [ ]:
lit_df

In [ ]:
lex_df.columns = [voc.replace('_', ' ') for voc in lex_df.columns]
vocab = [v.replace('_', ' ') for v in vocab]
lit_lex_df = lit_df[vocab].copy() #select columns with terms in lexical vocabulary
lit_lex_df

In [ ]:
mean = lit_lex_df.sum(axis=0) / lit_lex_df.astype(bool).sum(axis=0)
mean

In [ ]:
lit_lex_tfidf = lex_df[vocab].mul(mean, axis = 1)

In [ ]:
lit_lex_tfidf

In [ ]:
lit_lex_tfidf['weighted'] = lit_lex_tfidf[vocab].sum(axis=1, numeric_only = True)

In [ ]:
lit_lex_tfidf

In [ ]:
lit_lex_tfidf = lit_lex_tfidf.loc[lit_lex_tfidf.sum(axis=1) > 0]

In [ ]:
lex2 = pd.merge(cat_df, lit_lex_tfidf['weighted'], on = 'id_text', how = 'inner')
lex2 = pd.merge(lex2, lex[['length', 'n_matches', 'id_text']], on = 'id_text', how = 'inner')

Instead of dividing by length look at mean value of weighted
```python
lex2['norm'] = lex2['weigthed'] / lex2[vocab].astype(bool).sum(axis = 1)
```

In [ ]:
#lex2['norm'] = lex2['weighted'] / lex2['n_matches']
lex2['norm'] = lex2['weighted'] / lex2['length']
lex2.sort_values(by = 'norm', ascending = False)

In [ ]:
anchor = '<a href="http://oracc.org/dcclt/{}", target="_blank">{}</a>'
lex3 = lex2.copy()
lex3['id_text'] = [anchor.format(val,val) for val in lex2['id_text']]

In [ ]:
@interact(sort_by = lex3.columns, rows = (1, len(lex3), 1), min_length = (1,500,5))
def sort_df(sort_by = "weighted", ascending = False, rows = 25, min_length = 200):
    return lex3.loc[lex3.length >= min_length].sort_values(by = sort_by, ascending = ascending).reset_index(drop=True)[:rows].style

# OLD

In [ ]:
def make_ngrams(lemmas):
    lemmas = lemmas.split()
    lemmas_bi = bigrams(lemmas)
    lemmas_tri = trigrams(lemmas)
    lemmas_n = list(lemmas_bi) + list(lemmas_tri)
    lemmas_n = ['_'.join(lem) for lem in lemmas_n]
    lemmas = set(lemmas + lemmas_n)
    lemmas = [lem for lem in lemmas if not '[na]na' in lem]
    lit_vocab.extend(lemmas)
    return

In [ ]:
lit_vocab = []
lit_lines['lemma'].progress_apply(make_ngrams)
lit_vocab = list(set(lit_vocab))
lit_vocab.sort()
lit_vocab[:25]

Make ngrams: unigrams, bigrams, and trigrams. Represent bigrams and trigrams as MWEs, connected by underscores. Create a full list of all lemmas and ngrams, omitting all non-lemmatized words (or ngrams that include non-lemmatized words).

> Note: This step can be done with Countvectorizer, with setting ngrams = (1,3). Disadvantages of that approach:
> - we don not need a full DTM for the literary corpus
> - the DTM should be made on *lines* instead of *documents* to prevent words from consecutive lines to form bigrams or trigrams. Afterwards use groupby and agg to make DTM on document level
